In [ ]:
import sys
import os

# Ensure the directory exists
model_save_path = "models"
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist


# Manually set the project root directory (adjust if needed)
project_root = os.path.abspath("..")  # Moves up one level to project root

# Add the project directory to sys.path
sys.path.append(project_root)

# Now import the GRU model
from Models.GRU import GRUModel

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from Models.GRU import GRUModel   # Import the GRU model
from Training.PyTorchModular import train_model
import torch.nn as nn
from sklearn.metrics import mean_squared_error


import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset

# Import modularized functions
from dataPreprocessing import load_data, prepare_dataloader
from PyTorchModular import optuna_tune_and_train,  evaluate_model
from Models.GRU import GRUModel   

Load and preprocess data & Train the Model with Optuna

In [ ]:

# Load and preprocess data
train_file = "../Data/Train/trains1990s.csv"
config = {"use_fft": False, "use_exog": False}

(X_train_seq, y_train_seq, X_valid_seq, y_valid_seq, X_test_seq, y_test_seq, observation_dates, y_scaler) = load_data(
    train_file, sequence_length=12, config=config
)

X_train_seq = X_train_seq.reshape(X_train_seq.shape[0], X_train_seq.shape[1], 1)
X_valid_seq = X_valid_seq.reshape(X_valid_seq.shape[0], X_valid_seq.shape[1], 1)

y_train_seq = y_train_seq.reshape(-1, 1)
y_valid_seq = y_valid_seq.reshape(-1, 1)

batch_size = 32
train_loader = prepare_dataloader(X_train_seq, y_train_seq, batch_size=batch_size)
val_loader = prepare_dataloader(X_valid_seq, y_valid_seq, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run Optuna & Train Model
model, metadata = optuna_tune_and_train(
    model_class=GRUModel, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device,
    max_epochs=50,
    model_save_path="models",
    model_name="GRU_inflation",
    use_best_hyperparams=False,  # Set to False to force Optuna tuning every time
    n_trials=20
)


Make Predictions on Validation Set

In [ ]:
# Evaluate Model
df_comparison, rmse = evaluate_model(
    model=model, 
    val_loader=val_loader, 
    y_scaler=y_scaler, 
    observation_dates=observation_dates, 
    device=device
)